# Advanced Querying Mongo

Importing libraries and setting up connection

In [6]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [29]:
query = {'name':'Babelgum'}
db1 = list(db["companies"].find((query),{'name':1, '_id':0}))



### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [30]:
query2 = {'number_of_employees':{"$gt":5000}}
db2 = list(db["companies"].find(query2).sort('number_of_employees',-1).limit(20))

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [34]:
# Your Code
query3 = {'$and':[{'founded_year':{'$gte':2000}},{'founded_year':{'$lte':2005}}]}
db3 = list(db["companies"].find((query3),{'_id':0,'name':1,'founded_year':1}))


3734

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [37]:
query4 = {'$and':[{'founded_year':{'$lt':2010}},{"ipo.valuation_amount":{'$gt':100000000}}]}
db4 = list(db["companies"].find((query4),{'_id':0,'name':1,'ipo':1}))


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [41]:
query5 = {'$and':[{'founded_year':{'$lt':2005}},{"number_of_employees":{'$lt':1000}}]}
db5 = list(db["companies"].find(query5).sort("number_of_employees",-1).limit(10))


### 6. All the companies that don't include the `partners` field.

In [49]:
query6 = {'$and':[{'partners':{'$exists':'True'}},{'partners':{'$size':0}}]}
db6 = list(db["companies"].find(query6))
len(db6)

18647

### 7. All the companies that have a null type of value on the `category_code` field.

In [43]:
query7 = {'category_code':{'$type':'null'}}
db7 = list(db["companies"].find(query7))

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [47]:
query8 = {'$and':[{'number_of_employees':{'$gt':100}},{'number_of_employees':{'$lt':1000}}]}
db8 = list(db["companies"].find((query8),({'_id':0,'name':1,'number_of_employees':1})))

### 9. Order all the companies by their IPO price in a descending order.

In [58]:
db9 = list(db["companies"].find({"ipo.valuation_amount":{'$exists':'True'}}).sort("ipo.valuation_amount",-1))

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [ ]:
db10 = list(db["companies"].find({'number_of_employees':{'$exists':'True'}}).sort('number_of_employees',-1).limit(10))

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [60]:
query11 = {'$and':[{'founded_month':{'$gt':6}},{'founded_month':{'$lte':12}}]}
db11 = list(db["companies"].find(query11).limit(1000))

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [61]:
query12 = {'$and':[{'founded_year':{'$lt':2000}},{"acquisition.price_amount":{'$gt':1000000}}]}
db12 = list(db["companies"].find(query12))

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [67]:
query13 = {"acquisition.acquired_year":{'$gt':2010}}
db13 = list(db["companies"].find((query13),({'_id':0,'name':1,'acquisition':1})).sort("acquisition.price_amount",-1))

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [71]:
query14 = {'founded_year':{'$exists':True,'$not':{'$size':0}}}
db14 = list(db["companies"].find((query14),({'_id':0,'name':1,'founded_year':1})).sort('founded_year',-1).limit(100))

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [73]:
query15 = {'founded_day':{'$lte':7}}
db15 = list(db["companies"].find(query15).sort('acquisition.price_amount',-1).limit(10))

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [75]:
query16 = {'$and':[{'category_code':'web'},{'number_of_employees':{'$gt':4000}}]}
db16 = list(db["companies"].find(query16).sort('number_of_employees',1))

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [76]:
query17 = {'$and':[{"acquisition.price_currency_code":'EUR'},{"acquisition.price_amount":{'$gt':10000000}}]}
db17 = list(db["companies"].find(query17))

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [77]:
query18 = {'$and':[{"acquisition.acquired_month":{'$gte':1}},{"acquisition.acquired_month":{'$lte':3}}]}
db18 = list(db["companies"].find((query18),({'_id':0,'name':1,'acquisition':1})).limit(10))

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [80]:
query19 = {'$and':[{"founded_year":{'$gte':2000}},{"founded_year":{'$lte':2010}},{"acquisition.acquired_year":{'$gt':2011}}]}
db19 = list(db["companies"].find(query19))

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code